In [ ]:
import re
import os
from pathlib import Path

In [118]:
mat_ext_sig_re = re.compile(
    r'''^((?:[^/]*/)*)?                 # Path
        ((?:[@+]\w+\.)*)                # Namespace
        (\w+)                           # Function name
        (?:\(([\w\s\{\}\@'",]*)\))?     # Optional Arguments
        $''', re.VERBOSE
)

mat_namespace_re_pat = re.compile(
    r'([@+]\w+)\.',                     # Namespace list
    re.VERBOSE        
)

mat_properties_re = re.compile(
    r'''^
    (\w+)                               # argument
    (?:[ \t]*\(([\s\d,\:]+)\))?         # size
    (?:[ \t]*(\w+))?                    # argument class
    (?:[ \t]*\{(.+)\})?                 # validators
    (?:[ \t]*\=[ \t]*
        (\w+(?:\([\w\s\{\}\@'",]*\))?)  # value / function and optional arguments
    )?                                  
    (?:\s+\#+[ \t]*(.*))?               # Comment
    $''', re.VERBOSE
)

mat_validators_re_pat = re.compile(
    r'''
    [ \t]*
        (\w+(?:\([\w\s\{\}\@'",]*\))?)
    [ \t]*,?
    ''', re.VERBOSE
)


In [119]:
def parse_autodoc_input(prompt:str, matlab_root_dir:str, src_path:str):

    (path, namespacePath, filename, arguments) = mat_ext_sig_re.match(prompt.strip()).groups()
    namespace = mat_namespace_re_pat.findall(namespacePath)

    basepath = Path(matlab_root_dir if matlab_root_dir else src_path)
    abspath = (basepath / Path(path or '.')).resolve()
    filepath = (abspath / '/'.join(namespace)).resolve()

    return (filepath, filename, arguments, abspath, namespace)

def parse_property(prompt:str):
    (arg, size, arg_class, validators, default, comment) = mat_properties_re.match(prompt.strip()).groups()
    if size:
        size = [dim.strip() for dim in size.split(',')]
    validator_list = mat_validators_re_pat.findall(validators) if validators else None
    return (arg, size, arg_class, validator_list, default, comment)


In [121]:
parse_property("arg2 (1,:) double {mustBeImData(Data), mustBeInRange(Data,0,255), mustBeMember(Interp,{'linear','cubic','spline'})} = uint8(0)  #This is")

('arg2',
 ['1', ':'],
 'double',
 ['mustBeImData(Data)',
  'mustBeInRange(Data,0,255)',
  "mustBeMember(Interp,{'linear','cubic','spline'})"],
 'uint8(0)',
 'This is')

In [122]:
arg = '../@package.+main.test(arg1, arg2)'
matlab_root_dir = 'C:/Users'
src_path = os.getcwd()

parse_autodoc_input(arg, matlab_root_dir, src_path)

(WindowsPath('C:/@package/+main'),
 'test',
 'arg1, arg2',
 WindowsPath('C:/'),
 ['@package', '+main'])